In [0]:
# Define o caminho do arquivo
file_path = 'dbfs:/Volumes/workspace/default/veiculos/dados_veiculos_2025.csv'

# Lê o arquivo CSV como DataFrame Spark, com opções de cabeçalho, inferência de schema, delimitador e encoding
df = (
    spark.read.format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('delimiter', ';')
    .option('encoding', 'ISO-8859-1')
    .load(file_path)
)

from pyspark.sql.functions import regexp_replace, col, count, avg, sum, udf
from pyspark.sql.types import StringType

# Converte colunas de valor para float (substitui vírgula por ponto)
df = (
    df.withColumn('Valor_Venda', regexp_replace('Valor_Venda', ',', '.').cast('float'))
      .withColumn('Valor_Compra', regexp_replace('Valor_Compra', ',', '.').cast('float'))
)

# Calcula lucro por veículo
df = df.withColumn('Lucro', col('Valor_Venda') - col('Valor_Compra'))

# Agrupa por Ano e calcula as métricas
df_ano = (
    df.groupBy('Ano')
      .agg(
          count('*').alias('Total_Veiculos_Vendidos'),           # Total de veículos vendidos
          avg('Valor_Venda').alias('Valor_Medio_Venda'),         # Valor médio de venda
          avg('Lucro').alias('Lucro_Medio_Por_Veiculo'),         # Lucro médio por veículo
          sum('Lucro').alias('Lucro_Total')                      # Lucro total
      )
)

# Função para formatar valores como moeda brasileira (R$ 1.234,56)
def formatar_moeda(valor):
    if valor is None:
        return None
    return f"R$ {valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

formatar_moeda_udf = udf(formatar_moeda, StringType())

# Aplica a formatação nas colunas de valores
df_ano_formatado = (
    df_ano
    .withColumn("Valor_Medio_Venda", formatar_moeda_udf(col("Valor_Medio_Venda")))
    .withColumn("Lucro_Medio_Por_Veiculo", formatar_moeda_udf(col("Lucro_Medio_Por_Veiculo")))
    .withColumn("Lucro_Total", formatar_moeda_udf(col("Lucro_Total")))
)

display(df_ano_formatado)

Ano,Total_Veiculos_Vendidos,Valor_Medio_Venda,Lucro_Medio_Por_Veiculo,Lucro_Total
2018,2093,"R$ 70.262,99","R$ 10.339,52","R$ 21.640.612,79"
2006,2138,"R$ 70.043,16","R$ 10.483,24","R$ 22.413.159,86"
2000,2082,"R$ 70.848,26","R$ 10.637,50","R$ 22.147.276,62"
2007,2092,"R$ 71.960,59","R$ 10.825,50","R$ 22.646.940,61"
2013,2067,"R$ 70.207,25","R$ 10.379,43","R$ 21.454.279,91"
2005,2077,"R$ 70.205,73","R$ 10.485,49","R$ 21.778.360,74"
2009,2137,"R$ 69.886,44","R$ 10.462,44","R$ 22.358.233,63"
2008,2043,"R$ 69.834,19","R$ 10.219,76","R$ 20.878.978,03"
2019,2081,"R$ 70.825,05","R$ 10.565,47","R$ 21.986.736,10"
2021,2151,"R$ 69.882,35","R$ 10.479,83","R$ 22.542.107,41"
